In [1]:
# --- LIBRERIAS ---

import pandas as pd
import glob
import os

In [2]:
# --- CONFIGURACIONES MODIFICABLES ---

# 1. RUTA A LOS DATOS
# Busca todos los archivos que terminen en '*_COMPENSADA.xlsx' en la carpeta 
# especificada y los almacena en una lista
data_path = '../../data/raw/piezometers/'
piezometer_files = glob.glob(os.path.join(data_path, '*_COMPENSADA.xlsx'))

# Imprime los archivos encontrados
print(f'Archivos con datos piezometricos encontrados: {len(piezometer_files)}')
print(piezometer_files)

# 2. NOMBRES DE COLUMNAS
# Define un diccionario con nombres de columnas originales y a renombrar
columns_dict = {
    'TEMPERATURE' : 'Temperature_C',
    'NE_m' : 'Depth_m',
    'Cota_m' : 'Static_level_masl'
}

# Define una lista con con nombres de columnas no necesarias
columns_list = [
    'ms',
    'LEVEL',
    'P_baro'
]

Archivos con datos piezometricos encontrados: 6
['../../data/raw/piezometers\\Data_08_11_2024_SDH2PS01_COMPENSADA.xlsx', '../../data/raw/piezometers\\Data_15_07_2025_ SDH1PS01_COMPENSADA.xlsx', '../../data/raw/piezometers\\Data_15_07_2025_ SDH1PS02_COMPENSADA.xlsx', '../../data/raw/piezometers\\Data_15_07_2025_SDH2PP01_COMPENSADA.xlsx', '../../data/raw/piezometers\\Data_15_07_2025_SDH2PS02_COMPENSADA.xlsx', '../../data/raw/piezometers\\Data_15_07_2025_SDH2PS03_COMPENSADA.xlsx']


In [3]:
# --- DEFINICION DE FUNCIONES ---

# 1. MANEJO DE FECHAS E INDICE

def timestamp_as_index(df):
    """Convierte las columnas 'Date' y 'Time' en un indice Datetime"""

    # Copia el df original
    df_copy = df.copy()

    # Convierte las columnas 'Date' y 'Time' en strings
    date_str = df_copy['Date'].astype(str)
    time_str = df_copy['Time'].astype(str)

    # Crea la columna 'Timestamps' con formato datetime a partir de 'Date' y 'Time'
    df_copy['Timestamps'] = pd.to_datetime(
        date_str + ' ' + time_str,
        format='%Y-%m-%d %H:%M:%S'
    )

    # Establece 'Timestamps' como indice
    df_copy = df_copy.set_index('Timestamps')

    # Elimina las columnas 'Date' y 'Time'
    df_copy = df_copy.drop(columns=['Date', 'Time'])

    print('Indice datetime establecido')
    return df_copy


# 2. FORMATEO DE COLUMNAS

def format_columns(df):
    """Renombra columnas y elimina las innecesarias"""
    df_formatted = df.rename(columns=columns_dict).drop(columns=columns_list, axis=1, errors='ignore')
    print('Nombres de columnas formateados')
    return df_formatted

In [4]:
# --- BUCLE DE PROCESAMIENTO ---

# Define una carpeta de salida para los datos limpios y la crea si no existe
output_path = '../../data/processed/cleaned/'
os.makedirs(output_path, exist_ok=True)

# Bucle externo: itera sobre cada pozo
# file_path refiere a cada valor de la lista piezometer_files
for file_path in piezometer_files:
    
    # Define el nombre de cada pozo usando el nombre de archivo.
    # Mantiene la penultima cadena de texto, separadas por guion bajo
    base_name = os.path.basename(file_path)
    well_name = base_name.split('_')[-2].strip()
    
    # Imprime un titulo que indica el nombre del pozo y de archivo
    print("\n" + "="*80)
    print(f"POZO: {well_name}")
    print(f"Archivo: {os.path.basename(file_path)}")
    print("="*80)
 
    try:
        # Imprime un subtitulo y lee el archivo
        print("\n--- Lectura y formateo de datos ---")
        df_raw = pd.read_excel(file_path)
        
        # Aplica las funciones de formateo de fechas y columnas
        df_cleaned = timestamp_as_index(df_raw)
        df_cleaned = format_columns(df_cleaned)

    # En caso de error, avisa cual archivo no se pudo procesar y continua el bucle  
    except:
        print(f"ERROR: No se pudo procesar el archivo {file_path}")
        continue

    # Genera un nombre de archivo y asigna ubicacion al dataframe limpio
    output_filename = os.path.join(output_path, f'{well_name}_cleaned.csv')
    
    # Exporta el df limpio como csv
    df_cleaned.to_csv(output_filename)

    # Imprime informacion sobre el nombre y rango de fechas del output
    print(f"\nDatos limpios para {well_name} guardados en: {output_filename}")
    print(f"Rango de fechas: {df_cleaned.index.min()} - {df_cleaned.index.max()}")


POZO: SDH2PS01
Archivo: Data_08_11_2024_SDH2PS01_COMPENSADA.xlsx

--- Lectura y formateo de datos ---


Indice datetime establecido
Nombres de columnas formateados

Datos limpios para SDH2PS01 guardados en: ../../data/processed/cleaned/SDH2PS01_cleaned.csv
Rango de fechas: 2024-05-23 00:00:00 - 2024-11-08 16:25:00

POZO: SDH1PS01
Archivo: Data_15_07_2025_ SDH1PS01_COMPENSADA.xlsx

--- Lectura y formateo de datos ---
Indice datetime establecido
Nombres de columnas formateados

Datos limpios para SDH1PS01 guardados en: ../../data/processed/cleaned/SDH1PS01_cleaned.csv
Rango de fechas: 2024-05-23 00:00:00 - 2025-07-15 10:10:00

POZO: SDH1PS02
Archivo: Data_15_07_2025_ SDH1PS02_COMPENSADA.xlsx

--- Lectura y formateo de datos ---
Indice datetime establecido
Nombres de columnas formateados

Datos limpios para SDH1PS02 guardados en: ../../data/processed/cleaned/SDH1PS02_cleaned.csv
Rango de fechas: 2024-11-10 16:00:00 - 2025-07-15 10:00:00

POZO: SDH2PP01
Archivo: Data_15_07_2025_SDH2PP01_COMPENSADA.xlsx

--- Lectura y formateo de datos ---
Indice datetime establecido
Nombres de columnas forma